In [28]:
# install.packages("elmNNRcpp")
# install.packages("mltools")
# install.packages("caret", dependencies = TRUE)

library(elmNNRcpp)
library(mltools)
library(data.table)
library(caret)


In [29]:
set.seed(1234)


In [30]:
iris2 <- scale(iris[, 1:4])
iris2 <- as.data.frame(iris2)

iris2$Species <- as.factor(iris$Species)

head(iris2)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-0.8976739,1.01560199,-1.335752,-1.311052,setosa
2,-1.1392005,-0.13153881,-1.335752,-1.311052,setosa
3,-1.3807271,0.32731751,-1.392399,-1.311052,setosa
4,-1.5014904,0.09788935,-1.279104,-1.311052,setosa
5,-1.0184372,1.24503015,-1.335752,-1.311052,setosa
6,-0.5353840,1.93331463,-1.165809,-1.048667,setosa


In [31]:
particao <- createDataPartition(seq_len(dim(iris2)[1]), p = 0.7)
iris2_train <- iris2[particao$Resample1, ]
iris2_test <- iris2[-particao$Resample1, ]

dim(iris2_train)


[1] 106   5

In [32]:
x <- iris2_train[, 1:4]
y <- iris2_train[, 5]


In [33]:
# Scaling the data
x <- scale(x)

# One hot encoding the target variable
y <- one_hot(one_hot(as.data.table(y)))


In [34]:
# Converting to matrix, as required by the elmNNRcpp package

x <- as.matrix(x)

y <- as.matrix(y)


In [35]:
# modelo <- elm_train(x, y, nhid = 1000, actfun = "relu", init_weights = "uniform_negative", bias = TRUE, verbose = TRUE) # 0.849

modelo <- elm_train(x, y, nhid = 100, actfun = "sig", init_weights = "normal_gaussian", bias = TRUE, verbose = TRUE) # 0.909

Input weights will be initialized ...
Dot product of input weights and data starts ...
Bias will be added to the dot product ...
'sig' activation function will be utilized ...
The computation of the Moore-Pseudo-inverse starts ...
The computation is finished!

Time to complete : 0.19491 secs 


In [36]:
iris2_test_species <- as.factor(iris2_test$Species)

iris2_test <- scale(iris2_test[, 1:4])

iris2_test <- as.matrix(iris2_test)


In [37]:
result  <- elm_predict(modelo, iris2_test,normalize = TRUE)

In [38]:
result <- as.data.frame(result)

head(result)

,V1,V2,V3
,<dbl>,<dbl>,<dbl>
1,0.9297841,0.03713373,0.033082209
2,0.9737013,0.01945053,0.006848205
3,0.8692898,0.06455008,0.066160143
4,0.8793131,0.05332745,0.067359439
5,0.9843306,0.01031930,0.005350114
6,0.8712783,0.06709510,0.061626576


In [39]:
names(result) <- c("setosa", "versicolor", "virginica")

result$class <- colnames(result)[max.col(result, "first")]

head(result)

,setosa,versicolor,virginica,class
,<dbl>,<dbl>,<dbl>,<chr>
1,0.9297841,0.03713373,0.033082209,setosa
2,0.9737013,0.01945053,0.006848205,setosa
3,0.8692898,0.06455008,0.066160143,setosa
4,0.8793131,0.05332745,0.067359439,setosa
5,0.9843306,0.01031930,0.005350114,setosa
6,0.8712783,0.06709510,0.061626576,setosa


In [40]:
confusao <- table(result$class, iris2_test_species)

confusao

            iris2_test_species
             setosa versicolor virginica
  setosa         16          0         0
  versicolor      0          9         2
  virginica       1          1        15

In [41]:
taxa_acerto <- sum(diag(confusao))/sum(confusao)

taxa_acerto

[1] 0.9090909